# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [130]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

###################
## Cleaning data ##
###################

# Cities dataset #

big_four = ['NFL', 'MLB', 'NBA', 'NHL']
# Clean team strings in cities dataset:
for league in big_four:
    # Remove extensions with brackets:
    cities[league] = cities[league].str.replace('\[.+\]', '')
    # Split team names:
    cities[league] = cities[league].str.split('(?<=[a-z])(?=[A-Z])') #split between lowercase followed by uppercase
    

# NHL dataset #

# Keep only 2018 data:
nhl_df = nhl_df[nhl_df['year'] == 2018]
# Remove rows with divisions:
nhl_df = nhl_df[~nhl_df['GP'].str.contains('Division')]
# Remove * from team names
nhl_df['team'] = nhl_df['team'].str.replace('\*', '')


########################
## Values of interest ##
########################

## NHL dataset ##

# Convert numbers to integers:
nhl_df['W'] = nhl_df['W'].astype('int64')
nhl_df['L'] = nhl_df['L'].astype('int64')
# Calculate win-loss ratio:
nhl_df['win-loss'] = nhl_df['W']/(nhl_df['W'] + nhl_df['L'])
# Keep only team name and win-loss data:
nhl_df = nhl_df[['team', 'win-loss']]
# Keep only last name of team to match cities dataframe:
nhl_df['team'] = nhl_df['team'].apply(lambda x: re.search('\w*$', x).group(0)) # the group method returns the founs string

## Cities dataset ##

# Keep only NHL data and explode on teams:
cities = cities[['Metropolitan area', 'Population (2016 est.)[8]', 'NHL']]
cities = cities.explode('NHL') # convert lists in rows to one replicated row for each entry
# Rename columns for merge:
cities.rename(columns={'Population (2016 est.)[8]': 'Population', 'NHL': 'team'}, inplace=True)
# Keep only last name of team to match NHL dataframe:
cities['team'] = cities['team'].apply(lambda x: re.search('\w*$', x).group(0)) # the group method returns the found string


## Merging Data ##    

def mean_and_pop(group):
    mean = group['win-loss'].mean(axis=0)
    pop = group['Population'].iloc[0]
    row = pd.DataFrame({'WL': [mean], 'Pop': [pop]})
    return row

# Merging on team names:
nhl_merge = pd.merge(cities, nhl_df, on='team' , how='inner')
# Drop team column since it won't be used anymore:
nhl_merge.drop('team', axis=1, inplace=True)
# Group by area and apply mean, keeping same order and also returning the population:
nhl_merge = nhl_merge.groupby('Metropolitan area', sort=False).apply(mean_and_pop)
# Remove unused multi-index on result from groupby:
nhl_merge.index = nhl_merge.index.droplevel(1)
# Set type of population:
nhl_merge['Pop'] = nhl_merge['Pop'].astype('int64')


def nhl_correlation(): 
    
    population_by_region = nhl_merge['Pop'].array # pass in metropolitan area population from cities
    win_loss_by_region = nhl_merge['WL'].array # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [159]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]


###################
## Cleaning data ##
###################

# Cities dataset #

big_four = ['NFL', 'MLB', 'NBA', 'NHL']
# Clean team strings in cities dataset:
for league in big_four:
    # Remove extensions with brackets:
    cities[league] = cities[league].str.replace('\[.+\]', '')
    # Split team names:
    cities[league] = cities[league].str.split('(?<=[a-z])(?=[A-Z])') #split between lowercase followed by uppercase
    

# NBA dataset #

# Keep only 2018 data:
nba_df = nba_df[nba_df['year'] == 2018]
# Remove final characters from team names
nba_df['team'] = nba_df['team'].str.replace('\*.*$', '')
nba_df['team'] = nba_df['team'].str.replace(r'\xa0', '') #remove \xa0 as a literal expression
nba_df['team'] = nba_df['team'].str.replace('\(.*$', '') #now remove the rest of parenthesis and content at the end


########################
## Values of interest ##
########################

## NBA dataset ##

# Convert numbers to integers:
nba_df['W'] = nba_df['W'].astype('int64')
nba_df['L'] = nba_df['L'].astype('int64')
# Calculate win-loss ratio:
nba_df['win-loss'] = nba_df['W']/(nba_df['W'] + nba_df['L'])
# Keep only team name and win-loss data:
nba_df = nba_df[['team', 'win-loss']]
# Keep only last name of team to match cities dataframe:
nba_df['team'] = nba_df['team'].apply(lambda x: re.search('\w*$', x).group(0)) # the group method returns the founs string

## Cities dataset ##

# Keep only NBA data and explode on teams:
cities = cities[['Metropolitan area', 'Population (2016 est.)[8]', 'NBA']]
cities = cities.explode('NBA') # convert lists in rows to one replicated row for each entry
# Rename columns for merge:
cities.rename(columns={'Population (2016 est.)[8]': 'Population', 'NBA': 'team'}, inplace=True)
# Keep only last name of team to match NBA dataframe:
cities['team'] = cities['team'].apply(lambda x: re.search('\w*$', x).group(0)) # the group method returns the found string


## Merging Data ##    

def mean_and_pop(group):
    mean = group['win-loss'].mean(axis=0)
    pop = group['Population'].iloc[0]
    row = pd.DataFrame({'WL': [mean], 'Pop': [pop]})
    return row

# Merging on team names:
nba_merge = pd.merge(cities, nba_df, on='team' , how='inner')
# Drop team column since it won't be used anymore:
nba_merge.drop('team', axis=1, inplace=True)
# Group by area and apply mean, keeping same order and also returning the population:
nba_merge = nba_merge.groupby('Metropolitan area', sort=False).apply(mean_and_pop)
# Remove unused multi-index on result from groupby:
nba_merge.index = nba_merge.index.droplevel(1)
# Set type of population:
nba_merge['Pop'] = nba_merge['Pop'].astype('int64')



def nba_correlation():
    
    population_by_region = nba_merge['Pop'].array # pass in metropolitan area population from cities
    win_loss_by_region = nba_merge['WL'].array # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [169]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]


###################
## Cleaning data ##
###################

# Cities dataset #

big_four = ['NFL', 'MLB', 'NBA', 'NHL']
# Clean team strings in cities dataset:
for league in big_four:
    # Remove extensions with brackets:
    cities[league] = cities[league].str.replace('\[.+\]', '')
    # Split team names:
    cities[league] = cities[league].str.split('(?<=[a-z])(?=[A-Z])') #split between lowercase followed by uppercase
    

# MLB dataset #

# Keep only 2018 data:
mlb_df = mlb_df[mlb_df['year'] == 2018]
# Team names are clean!


########################
## Values of interest ##
########################

## MLB dataset ##

# Convert numbers to integers:
mlb_df['W'] = mlb_df['W'].astype('int64')
mlb_df['L'] = mlb_df['L'].astype('int64')
# Calculate win-loss ratio:
mlb_df['win-loss'] = mlb_df['W']/(mlb_df['W'] + mlb_df['L'])
# Keep only team name and win-loss data:
mlb_df = mlb_df[['team', 'win-loss']]
# Keep only last name of team to match cities dataframe:
mlb_df['team'] = mlb_df['team'].apply(lambda x: re.search('\w*$', x).group(0)) # the group method returns the founs string

## Cities dataset ##

# Keep only MLB data and explode on teams:
cities = cities[['Metropolitan area', 'Population (2016 est.)[8]', 'MLB']]
cities = cities.explode('MLB') # convert lists in rows to one replicated row for each entry
# Rename columns for merge:
cities.rename(columns={'Population (2016 est.)[8]': 'Population', 'MLB': 'team'}, inplace=True)
# Keep only last name of team to match MLB dataframe:
cities['team'] = cities['team'].apply(lambda x: re.search('\w*$', x).group(0)) # the group method returns the found string


## Merging Data ##    

def mean_and_pop(group):
    mean = group['win-loss'].mean(axis=0)
    pop = group['Population'].iloc[0]
    row = pd.DataFrame({'WL': [mean], 'Pop': [pop]})
    return row

# Merging on team names:
mlb_merge = pd.merge(cities, mlb_df, on='team' , how='inner')
# Drop team column since it won't be used anymore:
mlb_merge.drop('team', axis=1, inplace=True)
# Group by area and apply mean, keeping same order and also returning the population:
mlb_merge = mlb_merge.groupby('Metropolitan area', sort=False).apply(mean_and_pop)
# Remove unused multi-index on result from groupby:
mlb_merge.index = mlb_merge.index.droplevel(1)
# Set type of population:
mlb_merge['Pop'] = mlb_merge['Pop'].astype('int64')



def mlb_correlation(): 
    
    population_by_region = mlb_merge['Pop'].array # pass in metropolitan area population from cities
    win_loss_by_region = mlb_merge['WL'].array # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [181]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]



###################
## Cleaning data ##
###################

# Cities dataset #

big_four = ['NFL', 'MLB', 'NBA', 'NHL']
# Clean team strings in cities dataset:
for league in big_four:
    # Remove extensions with brackets:
    cities[league] = cities[league].str.replace('\[.+\]', '')
    # Split team names:
    cities[league] = cities[league].str.split('(?<=[a-z])(?=[A-Z])') #split between lowercase followed by uppercase
    

# NFL dataset #

# Keep only 2018 data:
nfl_df = nfl_df[nfl_df['year'] == 2018]
# Remove rows with divisions:
nfl_df = nfl_df[~nfl_df['W'].str.contains('^[AN]FC')]
# Remove * and + from end of team names:
nfl_df['team'] = nfl_df['team'].str.replace('\*|\+', '')


########################
## Values of interest ##
########################

## NFL dataset ##

# Convert numbers to integers:
nfl_df['W'] = nfl_df['W'].astype('int64')
nfl_df['L'] = nfl_df['L'].astype('int64')
# Calculate win-loss ratio:
nfl_df['win-loss'] = nfl_df['W']/(nfl_df['W'] + nfl_df['L'])
# Keep only team name and win-loss data:
nfl_df = nfl_df[['team', 'win-loss']]
# Keep only last name of team to match cities dataframe:
nfl_df['team'] = nfl_df['team'].apply(lambda x: re.search('\w*$', x).group(0)) # the group method returns the founs string

## Cities dataset ##

# Keep only NFL data and explode on teams:
cities = cities[['Metropolitan area', 'Population (2016 est.)[8]', 'NFL']]
cities = cities.explode('NFL') # convert lists in rows to one replicated row for each entry
# Rename columns for merge:
cities.rename(columns={'Population (2016 est.)[8]': 'Population', 'NFL': 'team'}, inplace=True)
# Keep only last name of team to match nfl dataframe:
cities['team'] = cities['team'].apply(lambda x: re.search('\w*$', x).group(0)) # the group method returns the found string


## Merging Data ##    

def mean_and_pop(group):
    mean = group['win-loss'].mean(axis=0)
    pop = group['Population'].iloc[0]
    row = pd.DataFrame({'WL': [mean], 'Pop': [pop]})
    return row

# Merging on team names:
nfl_merge = pd.merge(cities, nfl_df, on='team' , how='inner')
# Drop team column since it won't be used anymore:
nfl_merge.drop('team', axis=1, inplace=True)
# Group by area and apply mean, keeping same order and also returning the population:
nfl_merge = nfl_merge.groupby('Metropolitan area', sort=False).apply(mean_and_pop)
# Remove unused multi-index on result from groupby:
nfl_merge.index = nfl_merge.index.droplevel(1)
# Set type of population:
nfl_merge['Pop'] = nfl_merge['Pop'].astype('int64')



def nfl_correlation(): 
    
    population_by_region = nfl_merge['Pop'].array # pass in metropolitan area population from cities
    win_loss_by_region = nfl_merge['WL'].array # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [210]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]


###################
## Cleaning data ##
###################

# Cities dataset #

big_four = ['NFL', 'MLB', 'NBA', 'NHL']
# Clean team strings in cities dataset:
for league in big_four:
    # Remove extensions with brackets:
    cities[league] = cities[league].str.replace('\[.+\]', '')
    # Split team names:
    cities[league] = cities[league].str.split('(?<=[a-z])(?=[A-Z])') #split between lowercase followed by uppercase


# NHL dataset #

# Keep only 2018 data:
nhl_df = nhl_df[nhl_df['year'] == 2018]
# Remove rows with divisions:
nhl_df = nhl_df[~nhl_df['GP'].str.contains('Division')]
# Remove * from team names
nhl_df['team'] = nhl_df['team'].str.replace('\*', '')

    
# NBA dataset #

# Keep only 2018 data:
nba_df = nba_df[nba_df['year'] == 2018]
# Remove final characters from team names
nba_df['team'] = nba_df['team'].str.replace('\*.*$', '')
nba_df['team'] = nba_df['team'].str.replace(r'\xa0', '') #remove \xa0 as a literal expression
nba_df['team'] = nba_df['team'].str.replace('\(.*$', '') #now remove the rest of parenthesis and content at the end


# MLB dataset #

# Keep only 2018 data:
mlb_df = mlb_df[mlb_df['year'] == 2018]
# Team names are clean!


# NFL dataset #

# Keep only 2018 data:
nfl_df = nfl_df[nfl_df['year'] == 2018]
# Remove rows with divisions:
nfl_df = nfl_df[~nfl_df['W'].str.contains('^[AN]FC')]
# Remove * and + from end of team names:
nfl_df['team'] = nfl_df['team'].str.replace('\*|\+', '')



########################
## Values of interest ##
########################

## NHL dataset ##

# Convert numbers to integers:
nhl_df['W'] = nhl_df['W'].astype('int64')
nhl_df['L'] = nhl_df['L'].astype('int64')
# Calculate win-loss ratio:
nhl_df['NHL win-loss'] = nhl_df['W']/(nhl_df['W'] + nhl_df['L'])
# Keep only team name and win-loss data:
nhl_df = nhl_df[['team', 'NHL win-loss']]
# Keep only last name of team to match cities dataframe:
nhl_df['team'] = nhl_df['team'].apply(lambda x: re.search('\w*$', x).group(0)) # the group method returns the founs string
# Rename team column to merge with cities:
nhl_df.rename(columns={'team': 'NHL'}, inplace=True)


## NBA dataset ##

# Convert numbers to integers:
nba_df['W'] = nba_df['W'].astype('int64')
nba_df['L'] = nba_df['L'].astype('int64')
# Calculate win-loss ratio:
nba_df['NBA win-loss'] = nba_df['W']/(nba_df['W'] + nba_df['L'])
# Keep only team name and win-loss data:
nba_df = nba_df[['team', 'NBA win-loss']]
# Keep only last name of team to match cities dataframe:
nba_df['team'] = nba_df['team'].apply(lambda x: re.search('\w*$', x).group(0)) # the group method returns the founs string
# Rename team column to merge with cities:
nba_df.rename(columns={'team': 'NBA'}, inplace=True)


## MLB dataset ##

# Convert numbers to integers:
mlb_df['W'] = mlb_df['W'].astype('int64')
mlb_df['L'] = mlb_df['L'].astype('int64')
# Calculate win-loss ratio:
mlb_df['MLB win-loss'] = mlb_df['W']/(mlb_df['W'] + mlb_df['L'])
# Keep only team name and win-loss data:
mlb_df = mlb_df[['team', 'MLB win-loss']]
# Keep only last name of team to match cities dataframe:
mlb_df['team'] = mlb_df['team'].apply(lambda x: re.search('\w*$', x).group(0)) # the group method returns the founs string
# Rename team column to merge with cities:
mlb_df.rename(columns={'team': 'MLB'}, inplace=True)


## NFL dataset ##

# Convert numbers to integers:
nfl_df['W'] = nfl_df['W'].astype('int64')
nfl_df['L'] = nfl_df['L'].astype('int64')
# Calculate win-loss ratio:
nfl_df['NFL win-loss'] = nfl_df['W']/(nfl_df['W'] + nfl_df['L'])
# Keep only team name and win-loss data:
nfl_df = nfl_df[['team', 'NFL win-loss']]
# Keep only last name of team to match cities dataframe:
nfl_df['team'] = nfl_df['team'].apply(lambda x: re.search('\w*$', x).group(0)) # the group method returns the founs string
# Rename team column to merge with cities:
nfl_df.rename(columns={'team': 'NFL'}, inplace=True)


## Cities dataset ##

# Drop population data since we won't use here:
cities.drop('Population (2016 est.)[8]', axis=1, inplace=True)
# Explode all teams (the order doesn't matter):
for league in big_four: #use same list as at the beggining of the question
    cities = cities.explode(league) # convert lists in rows to one replicated row for each entry
    # Keep only last name of team to match nfl dataframe:
    cities[league] = cities[league].apply(lambda x: re.search('\w*$', x).group(0)) # the group method returns the found string


def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    ## Calculate hipothesis tests here! and fill p_values dataframe with the results! ##
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

cities.head()

,Metropolitan area,NFL,MLB,NBA,NHL
0,New York City,Giants,Yankees,Knicks,Rangers
0,New York City,Giants,Yankees,Knicks,Islanders
0,New York City,Giants,Yankees,Knicks,Devils
0,New York City,Giants,Yankees,Knicks,Rangers
0,New York City,Giants,Yankees,Knicks,Islanders
